# Julia 元编程

## 惊吓引号

Julia 支持*元编程*。这类似于符号编程，我们处理的是表达式（如 `$2+2$`），而不是值（如 `$4$`）。

通常，Julia 将我们提供的所有代码视为一组指令，并执行它们。如果我们输入 `2+2`，它将忠实地将这些数字相加并给出结果。

In [1]:
2+2

4

我们可以用引号来阻止这种情况发生。用 `"` 将代码括起来，会将其视为纯粹的字符序列，而根本不将其视为代码。

In [2]:
x = "2+2"

"2+2"

然后，我们可以明确地告诉 Julia 稍后对其进行求值。

In [3]:
eval(Meta.parse(x))

ast = Meta.parse(x)
# ast.args[1] = 3
ast

#s expresion
Meta.show_sexpr(ast)

(:call, :+, 2, 2)call, :+, 2, 2)

为什么要用这种复杂的方式来做加法呢？诀窍在于，当我们拥有表达式 `2+2` 时，我们可以用各种有趣的方式修改它。举个简单的例子，想象一下用 `-` 替换 `+`。

In [4]:
x = replace(x, "+"=>"-")

"2-2"

In [5]:
eval(Meta.parse(x))


0

我们实际上不想在这里使用字符串；Julia 有一种更强大的引用代码的方式，即皱眉运算符 `:()`。

In [6]:
x = :(2+2)

:(2 + 2)

In [7]:
eval(x)

4

我们可以引用更大的表达式，包括代码块和整个函数定义。关键字 `quote` 是 `begin` 的替代品，它返回被引用的代码块。

在较大的代码块中，Julia 会保留行号信息，这些信息会以注释的形式出现。

In [8]:
quote
  x = 2 + 2
  hypot(x, 5)
end

quote
    #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X21sZmlsZQ==.jl:2 =#
    x = 2 + 2
    #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X21sZmlsZQ==.jl:3 =#
    hypot(x, 5)
end

In [9]:
quote
    function mysum(xs)
      sum = 0
      for x in xs
        sum += x
      end
      return sum
    end
end

quote
    #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X22sZmlsZQ==.jl:2 =#
    function mysum(xs)
        #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X22sZmlsZQ==.jl:2 =#
        #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X22sZmlsZQ==.jl:3 =#
        sum = 0
        #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X22sZmlsZQ==.jl:4 =#
        for x = xs
            #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X22sZmlsZQ==.jl:5 =#
            sum += x
            #= /Users/shufanzh

In [10]:
:(function mysum(xs)
    sum = 0
    for x in xs
      sum += x
    end
  end)

:(function mysum(xs)
      #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X23sZmlsZQ==.jl:1 =#
      #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X23sZmlsZQ==.jl:2 =#
      sum = 0
      #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X23sZmlsZQ==.jl:3 =#
      for x = xs
          #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X23sZmlsZQ==.jl:4 =#
          sum += x
          #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X23sZmlsZQ==.jl:5 =#
      end
  end)

## 表达式树的果实

字符串支持“插值”，这使我们可以轻松地从较小的组件构建较大的字符串。

In [11]:
x = "yields falsehood when preceded by its quotation"
println(x)

yields falsehood when preceded by its quotation


In [12]:
y = "'$x' $x"
println(y)

'yields falsehood when preceded by its quotation' yields falsehood when preceded by its quotation


In [13]:
x = :(2+2)

:(2 + 2)

In [14]:
test = :($x * $x)

:((2 + 2) * (2 + 2))

In [15]:
eval(test)

16

## Eval 之根

`eval` 不仅仅可以返回结果。如果我们引用类似函数定义的东西会发生什么？

In [16]:
ex2 = :(ex = :(foo233() = println("I'm foo!")))

:(ex = $(Expr(:quote, :(foo233() = begin
          #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X36sZmlsZQ==.jl:1 =#
          println("I'm foo!")
      end))))

In [17]:
eval(ex2)

:(foo233() = begin
          #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X36sZmlsZQ==.jl:1 =#
          println("I'm foo!")
      end)

它实际上什么也没做；目前还没有。

In [18]:
foo233()  # throws UndefVarError

UndefVarError: UndefVarError: `foo233` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

但是对 `ex` 求值会使 `foo` 生效！

In [19]:
eval(ex)

foo233 (generic function with 1 method)

In [20]:
foo233()

I'm foo!


使用插值，我们可以动态构建函数定义；事实上，我们可以一次创建一系列函数。

In [21]:
for name in [:foo, :bar, :baz]
  println(:($name() = println($("I'm $(name)!"))))
end

foo() = begin
        #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X50sZmlsZQ==.jl:2 =#
        println("I'm foo!")
    end
bar() = begin
        #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X50sZmlsZQ==.jl:2 =#
        println("I'm bar!")
    end
baz() = begin
        #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X50sZmlsZQ==.jl:2 =#
        println("I'm baz!")
    end

        println("I'm foo!")
    end
bar() = begin
        #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X50sZmlsZQ==.jl:2 =#
        println("I'm bar!")
    end
baz() = begin
        #= /Users/shufanzhang/Documents/c

然后也可以使用 `eval` 使它们生效。

In [22]:
for name in [:foo, :bar, :baz]
  eval(:($name() = println($("I'm $(name)!"))))
end

In [23]:
bar()

I'm bar!


In [24]:
baz()

I'm baz!


在封装 API（例如，来自 C 库或通过 HTTP）时，这可能是一个*极其*有用的技巧。API 通常会定义可用函数的列表，因此您可以获取该列表并一次性生成整个封装器！有关示例，请参见 Clang.jl、TensorFlow.jl 或 Base 线性代数封装器。

## 原罪

这是一个更实际的例子。考虑以下基于泰勒级数的 `sin` 函数定义。

$$sin(x) = \sum_{k=1}^{\infty} \frac{(-1)^k}{(1+2k)!} x^{1+2k}$$

In [25]:
mysin(x) = sum((-1)^k/factorial(1+2k) * x^(1+2k) for k = 0:5)

mysin (generic function with 1 method)

In [26]:
mysin(0.5), sin(0.5)

(0.4794255386041834, 0.479425538604203)

为了了解我们目前所处的位置，我们将对其进行基准测试。

In [27]:
using BenchmarkTools
@benchmark mysin(0.5)

BenchmarkTools.Trial: 10000 samples with 997 evaluations per sample.
 Range (min … max):  21.231 ns … 66.073 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     24.240 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   24.510 ns ±  1.803 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂   ▃▂ ▁▂▁ ▃▇█▂▅▁█▇▄▃▆▃▄▁▅▃▄▁▂     ▃                        ▂
  █▇▅▇██▆███▆████████████████████▇█▇███▇▇▇▇█▇▆▆▇▇▇▆▆▆▅▆▆▆▆▆▅▅ █
  21.2 ns      Histogram: log(frequency) by time      31.5 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

目前，这比它可能的速度要慢得多。原因是我们在循环遍历 `k`，这相对昂贵。写出会快得多：

In [28]:
mysin(x) = x - x^3/6 + x^5/120 # + ...

mysin (generic function with 1 method)

但这写起来很乏味，而且不再像原始的泰勒级数了。更难判断我们是否犯了错误，而且我们很容易修改它。有没有两全其美的方法呢？

让 Julia 为我们写出那段代码怎么样？

首先，让我们考虑 `+` 函数的符号版本。

In [29]:
plus(a, b) = :($a + $b)

plus (generic function with 1 method)

In [30]:
a= 1
b = 2
a1 = $a
b1 = $b


ErrorException: syntax: "$" expression outside quote around /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y102sZmlsZQ==.jl:3

In [31]:
expp = :(1+2+3)
qq = :($expp+4+5+6)
println(qq)

ex = eval(qq)
println(ex)

(1 + 2 + 3) + 4 + 5 + 6
21


In [32]:
plus(1, 2)

:(1 + 2)

有了 `plus`，我们可以做更有趣的事情，比如符号 `sum`：

In [33]:
reduce(+, 1:10)

55

In [34]:
reduce(plus, 1:10)

:(((((((((1 + 2) + 3) + 4) + 5) + 6) + 7) + 8) + 9) + 10)

In [35]:
eval(ans)

鉴于此，我们也可以对符号变量求和。

In [36]:
reduce(plus, [:(x^2), :x, 1])

:((x ^ 2 + x) + 1)

这为我们提供了谜题的重要组成部分，但我们还需要弄清楚我们正在对*什么*求和。让我们创建上面泰勒级数的符号版本，它对 `k` 的值进行插值。

In [37]:
k = 2
:($((-1)^k) * x^$(1+2k) / $(factorial(1+2k)))

:((1 * x ^ 5) / 120)

现在我们有了一项，我们可以根据需要生成任意多项。

In [38]:
terms = [:($((-1)^k) * x^$(1+2k) / $(factorial(1+2k))) for k = 0:5]

6-element Vector{Expr}:
 :((1 * x ^ 1) / 1)
 :((-1 * x ^ 3) / 6)
 :((1 * x ^ 5) / 120)
 :((-1 * x ^ 7) / 5040)
 :((1 * x ^ 9) / 362880)
 :((-1 * x ^ 11) / 39916800)

然后对它们求和 –

In [39]:
reduce(plus, terms)

:((((((1 * x ^ 1) / 1 + (-1 * x ^ 3) / 6) + (1 * x ^ 5) / 120) + (-1 * x ^ 7) / 5040) + (1 * x ^ 9) / 362880) + (-1 * x ^ 11) / 39916800)

并从中创建一个函数定义：

In [40]:
:(mysin(x) = $terms)

:(mysin(x) = begin
          #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y124sZmlsZQ==.jl:1 =#
          Expr[:((1 * x ^ 1) / 1), :((-1 * x ^ 3) / 6), :((1 * x ^ 5) / 120), :((-1 * x ^ 7) / 5040), :((1 * x ^ 9) / 362880), :((-1 * x ^ 11) / 39916800)]
      end)

In [41]:
eval(ans)

In [42]:
mysin(0.5), sin(0.5)

(0.47942708333333334, 0.479425538604203)

In [43]:
@benchmark mysin(0.5)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations per sample.
 Range (min … max):  0.833 ns … 23.167 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     0.917 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   0.979 ns ±  0.487 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▅    █    ▃    ▇    ▄                       ▂    ▂     ▂
  ▆▁▁▁█▁▁▁▁█▁▁▁▁█▁▁▁▁█▁▁▁▁█▁▁▁█▇▁▁▁█▁▁▁▁▅▁▁▁▁▅▁▁▁▁█▁▁▁▁█▁▁▁▆ █
  0.833 ns     Histogram: log(frequency) by time     1.33 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

在我的机器上，`sin2` 大约需要 50 *纳*秒才能运行——对于一个朴素的实现来说还不错。如果我们向光子发起 20 米短跑挑战，我们会赢！

## 表达式之下

它实际上只是一个普通的树形数据结构，我们可以查看它的内部。

In [44]:
ex = :(f(x, y))
#ex = Expr(:call, :f, :x, :y)
@show ex.head;
@show ex.args;

ex.head = :call
ex.args = Any[:f, :x, :y]


在上面的示例中，我们在字符串中将 `+` 替换为 `-`。在表达式中，我们可以通过更改表达式的 `.args` 来做到这一点。

In [45]:
ex = :(2+2)
@show ex.head;
@show ex.args;


ex.head = :call
ex.args = Any[:+, 2, 2]


In [46]:
ex.args[1] = :-
ex

:(2 - 2)

In [47]:
eval(ex)

0

请注意，更大、更复杂的表达式比这要复杂一些。它们是*嵌套*的，这意味着表达式 `2+3` 包含在更大的表达式 `1 + (2 + 3)` 中。

In [48]:
ex = :(1 + (2 + 3))

:(1 + (2 + 3))

In [49]:
ex.args

3-element Vector{Any}:
  :+
 1
  :(2 + 3)

一个名为 MacroTools 的包提供了一种处理此问题的方法。它依次向我们显示*所有*子表达式，允许我们决定如何更改它们。可以将其视为类似于查找和替换操作。这是一个在表达式中查找所有整数并将其递增的示例。

In [50]:
# import Pkg; Pkg.add("MacroTools")
using MacroTools
using MacroTools: postwalk

In [51]:
# postwalk(ex)

In [52]:
postwalk(ex) do x
  x isa Integer ? x+1 : x
end
# This will traverse the expression `ex` and increment every integer it finds by 1.

:(2 + (3 + 4))

要了解正在发生的事情，您可以使用 `@show` 来查看 `postwalk` 看到了什么。

（`@show` 是 Julia 最有用的功能；如果您不理解代码在做什么，请将其放入并查看正在发生的事情。）

In [53]:
map(x -> @show(x), [1,2,3])

x = 1
x = 2
x = 3


3-element Vector{Int64}:
 1
 2
 3

In [54]:
postwalk(ex) do x
  @show x
end

x = :+
x = 1
x = :+
x = 2
x = 3
x = :(2 + 3)
x = :(1 + (2 + 3))


:(1 + (2 + 3))

MacroTools 还提供了用于对表达式进行*模式匹配*的工具。`a_ + b_` 充当模板；如果提供的表达式看起来像模板，则 `a` 和 `b` 将匹配相加的两个事物。如果不是，它们将只是 `nothing`。

In [55]:
ex = :(2 + 3)
@capture(ex, a_ + b_)

true

In [56]:
a, b

(2, 3)

In [57]:
ex = :(f(2,3))
@capture(ex, a_ + b_)

false

In [58]:
a, b

(nothing, nothing)

我们最终可以使用它来替换表达式中*所有*的 `+` 为 `-`，而不仅仅是一个。

In [59]:
ex = :(3x^2 + (2x + 1))

:(3 * x ^ 2 + (2x + 1))

In [60]:
postwalk(ex) do x
  @capture(x, a_ + b_) || return x
  :($a - $b)
end

:(3 * x ^ 2 - (2x - 1))

In [61]:
macro say_hello(name)
    return quote
            println("Hello, ", $(esc(name)))
        end
end

@say_hello (macro with 1 method)

In [62]:
@macroexpand @say_hello "Alice"


quote
    #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y166sZmlsZQ==.jl:3 =#
    Main.println("Hello, ", "Alice")
end

In [63]:
macro bad_macro(x)
    quote
        esc($x) = 42
    end
end

@bad_macro (macro with 1 method)

In [64]:
x = "Hello"
@bad_macro(x)
println(x)  

Hello


In [65]:
@macroexpand @bad_macro x

quote
    #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y201sZmlsZQ==.jl:3 =#
    var"#96#esc"(var"#97#x") = begin
            #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y201sZmlsZQ==.jl:3 =#
            42
        end
end


## 卫生宏 (Hygienic Macros) 和 `esc`



宏在默认情况下是*卫生的*。这意味着宏内部创建的变量不会与调用宏的代码中的变量发生冲突。这可以防止意外的名称冲突和错误。


In [66]:

macro hygienic_macro()
    quote
        local_var = "I am local to the macro"
        println(local_var)
    end
end

@hygienic_macro()
# println(local_var) # This would cause an UndefVarError because local_var is hygienic


I am local to the macro



在上面的例子中，`local_var` 是宏的局部变量，在宏外部不可见。

然而，有时您希望宏能够访问或修改调用者作用域中的变量。这就是 `esc` 的用途。`esc` (escape) 允许您将表达式标记为应该在调用者的作用域中进行求值，而不是在宏的卫生作用域中。


In [67]:

macro assign_to_caller(var_name, value)
    quote
        $(esc(var_name)) = $(esc(value))
    end
end

x = 10
@assign_to_caller(x, 20)
println("x after macro: ", x) # x will be 20

y = "hello"
@assign_to_caller(y, "world")
println("y after macro: ", y) # y will be "world"


x after macro: 20
y after macro: world


In [68]:

macro assign_to_caller_new(theX,value)
    quote
        $(esc(theX)) = $(esc(value))
    end
end

theX = 10
@assign_to_caller_new(theX,20)
println("theX after macro: ", theX) # x will be 20


theX after macro: 20



在这个例子中：
- `$(esc(var_name))` 确保 `var_name` (例如 `x`) 指的是调用者作用域中的变量 `x`，而不是宏内部的新变量。
- `$(esc(value))` 确保 `value` (例如 `20`) 在调用者的作用域中进行求值（如果它是一个表达式）。对于字面量如 `20` 或 `"world"`，`esc` 的效果不那么明显，但对于更复杂的表达式则很重要。

`esc` 对于编写需要与周围代码交互的宏至关重要，例如定义新变量或修改现有变量的宏。

需要注意的是，过度使用 `esc` 会破坏宏的卫生性，并可能导致难以调试的名称冲突。因此，只有在确实需要与调用者作用域交互时才应使用它。


## 宏攻击

您可能已经见过宏了——Julia 中的基本功能，如 `@show`、`@time` 和 `@inline` 实际上都是宏。对于基本用法，将它们视为改变代码运行方式的简单注释就足够了。但是我们现在已经足够了解它们在底层是如何工作的了。

普通函数从来看不到*代码*，只看到*值*。如果我们将 `2+2` 传递给函数，它会看到 `4`。

In [69]:
function foo(x)
  @show x
  return x
end

foo (generic function with 2 methods)

In [70]:
foo(2+2)

x = 4


4

宏非常像函数，但是它们看到的是传递给它们的*代码*，就像我们上面看到的表达式一样。宏有机会操作此代码并更改其行为方式。

In [71]:
macro foo(x)
  @show x
  return x
end

@foo (macro with 1 method)

In [72]:
@foo(2+2)

x = :(2 + 2)


4

举一个简单的例子，我们可以替换 `+` 的一个参数并得到不同的结果。

In [73]:
macro foo(x)
  x.args[2] = 5
  return x
end

@foo (macro with 1 method)

In [74]:
@foo(2+2)

7

MacroTools 提供了一个有用的工具 `@expand`，用于查看宏内部发生了什么；它揭示了宏返回的代码而不运行它。

In [75]:
@expand @foo(2+2)

:(5 + 2)

In [76]:
@macroexpand @foo(2+2)

:(5 + 2)

您当然可以在 Julia 自带的宏上使用它，这是学习它们工作原理的好方法。

In [77]:
@time 2+2

  0.000000 seconds


4

In [78]:
@expand @time 2+2

quote
    #= timing.jl:315 =#
    begin
        #= timing.jl:320 =#
        local fly = begin
                    #= timing.jl:574 =#
                    $(Expr(:meta, :force_compile))
                    #= timing.jl:575 =#
                    (Base.Threads).lock_profiling(true)
                    #= timing.jl:576 =#
                    local penguin = (Base.Threads).LOCK_CONFLICT_COUNT[]
                    #= timing.jl:577 =#
                    local locust = Base.gc_num()
                    #= timing.jl:578 =#
                    local grasshopper = Base.time_ns()
                    #= timing.jl:579 =#
                    Base.cumulative_compile_timing(true)
                    #= timing.jl:580 =#
                    local swan = Base.cumulative_compile_time_ns()
                    #= timing.jl:581 =#
                    local goosander = $(Expr(:tryfinally, :(2 + 2), quote
    grasshopper = Base.time_ns() - grasshopper
    #= timing.jl:583 =#
    Base.cumulative_compile_timin

让我们做一些更高级的事情。使用我们上面开发的技术，我们可以在代码中查找并替换 `+` 表达式。我们仍然会进行加法运算，但我们也会记录我们添加的内容以进行调试。

首先，让我们使用引号创建一个示例表达式。

In [79]:
ex = quote
  s = 0
  for x in xs
    s = s + x
  end
  return s
end

quote
    #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y226sZmlsZQ==.jl:2 =#
    s = 0
    #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y226sZmlsZQ==.jl:3 =#
    for x = xs
        #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y226sZmlsZQ==.jl:4 =#
        s = s + x
        #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y226sZmlsZQ==.jl:5 =#
    end
    #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y226sZmlsZQ==.jl:6 =#
    return s
end

我们可以使用 `ex` 来开发我们想要的代码转换，并检查代码是否做了正确的事情。

In [80]:
postwalk(ex) do x
  @capture(x, a_ + b_) || return x
  quote
    println("Adding " * string($a) * " to " * string($b))
    $x
  end
end

quote
    #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y226sZmlsZQ==.jl:2 =#
    s = 0
    #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y226sZmlsZQ==.jl:3 =#
    for x = xs
        #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y226sZmlsZQ==.jl:4 =#
        s = begin
                #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y231sZmlsZQ==.jl:4 =#
                println("Adding " * string(s) * " to " * string(x))
                #= /Users/shufanzhang/Documents/coderepos/Julia_Tutorials/more-advanced-materials/metaprogramming/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y231sZm

现在我们只需将其包装在一个宏中，就可以将其添加到普通的函数定义中了！

In [81]:
macro log_adds(ex)
  postwalk(ex) do x
    @capture(x, a_ + b_) || return x
    quote
      println("Adding " * string($a) * " to " * string($b))
      $x
    end
  end
end

@log_adds (macro with 1 method)

In [82]:
@log_adds function mysum(xs)
  sum = 0
  for x in xs
    sum = sum + x
  end
  return sum
end

mysum (generic function with 1 method)

In [83]:
mysum(1:10)

Adding 0 to 1
Adding 1 to 2
Adding 3 to 3
Adding 6 to 4
Adding 10 to 5
Adding 15 to 6
Adding 21 to 7
Adding 28 to 8
Adding 36 to 9
Adding 45 to 10
Adding 1 to 2
Adding 3 to 3
Adding 6 to 4
Adding 10 to 5
Adding 15 to 6
Adding 21 to 7
Adding 28 to 8
Adding 36 to 9
Adding 45 to 10


55

添加和删除 `@log_adds` 比手动插入调试调用要好得多，尤其是在代码中有很多 `+` 的情况下。也许您可以扩展宏以支持记录其他运算符，例如 `-`。

## 生成函数

_生成函数_是 Julia 独有的一种新的元编程工具。本节将简要描述和说明它们，[这篇博文](http://mikeinnes.github.io/2017/08/24/cudanative.html)为感兴趣的读者提供了更多示例。

从本质上讲，生成函数就像一个作用于*类型*而不是表达式树的宏。如果我们尝试检查参数，我们将看到它们的类型而不是它们的值。

In [84]:
@generated function gadd(a, b)
  Core.println("a = $a, b = $b")
  :(a+b)
end

gadd (generic function with 1 method)

In [85]:
gadd(5, 2.5)

a = Int64, b = Float64


7.5

当处理更复杂的类型时，生成函数会变得更加强大。例如，请注意数组如何在类型内部存储其秩，因此这是我们可以为其生成代码的内容。

In [86]:
rand(2,2)

2×2 Matrix{Float64}:
 0.330355  0.673685
 0.788324  0.325975

In [87]:
typeof(ans)

Nothing

这为什么有用呢？如果您编写处理数组的代码，您会注意到您通常需要为数组的每个维度编写一个 `for` 循环。如果您想处理一个 7D 数组，则需要编写七个嵌套循环！

In [88]:
function mysum(xs::Array{<:Any,1})
  sum = 0
  for i = 1:length(xs)
    sum += xs[i]
  end
  return sum
end

function mysum(xs::Array{<:Any,2})
  sum = 0
  for i = 1:size(xs,1)
    for j = 1:size(xs, 2)
      sum += xs[i]
    end
  end
  return sum
end

mysum (generic function with 3 methods)

其他语言只是为向量、矩阵以及可能的 3D 数组硬编码每个函数的一个版本，而 Julia 允许我们通过根据需要生成嵌套循环来轻松编写 N 维算法。


## 宏用于领域特定语言 (DSL)

宏是 Julia 中创建领域特定语言 (DSL) 的强大工具。DSL 允许您为特定问题域定制语法，使代码更具表现力和可读性。

通过宏，您可以转换自定义语法到标准的 Julia 代码。这使得为复杂系统（如优化模型、查询语言或并行计算框架）构建直观的接口成为可能。

例如，`JuMP.jl` 包使用宏来提供一种代数建模语言，用于定义优化问题，用户可以像写数学公式一样编写模型。

```julia
using JuMP, GLPK
model = Model(GLPK.Optimizer)
@variable(model, x >= 0)
@variable(model, y >= 0)
@objective(model, Min, 12x + 20y)
@constraint(model, c1, 6x + 8y >= 100)
@constraint(model, c2, 7x + 12y >= 120)
print(model) # 打印出数学公式形式的模型
optimize!(model)
```

在这个例子中，`@variable`、`@objective` 和 `@constraint` 都是宏，它们将这种特定领域的语法转换为 JuMP 包可以理解和处理的底层数据结构和函数调用。

创建 DSL 通常涉及设计一套宏来解析和转换用户提供的特定语法。这需要仔细考虑用户体验和生成的 Julia 代码的效率。



## 高级卫生和 `gensym`

虽然宏默认是卫生的，并且 `esc` 允许我们有选择地“破坏”卫生性以与调用者作用域交互，但有时我们需要在宏内部创建保证唯一的变量名，而这些变量名不应与用户代码中的任何现有或未来变量冲突，即使在 `esc` 的上下文中也是如此。

这就是 `gensym` 函数发挥作用的地方。`gensym`（generate symbol 的缩写）创建一个在当前模块中保证唯一的符号。这在编写复杂的宏时特别有用，这些宏需要在其生成的代码中引入临时变量，并且您希望避免任何意外的名称冲突。

```julia
macro advanced_hygiene_example()
    temp_var = gensym("internal_temp") # 创建一个唯一的符号，例如 :##internal_temp#123
    quote
        # 使用 esc 来在调用者作用域中操作，但内部变量名是唯一的
        $(esc(temp_var)) = "This is a hygienically generated temporary variable"
        println($(esc(temp_var)))
        # 假设我们还想在宏的局部作用域（如果需要）使用它，或者只是为了演示
        # local_unique_var = $(esc(temp_var)) # 这行可能没有实际意义，取决于宏的目标
    end
end

# 即使调用者作用域中有名为 internal_temp 的变量，也不会发生冲突
internal_temp = "This is from the caller scope"
@advanced_hygiene_example()
println("Caller scope internal_temp: ", internal_temp) # 仍然是 "This is from the caller scope"

# 尝试访问 gensym 创建的变量通常是不直接的，因为它的确切名称是生成的
# 例如，你不能直接写 ##internal_temp#123，因为数字后缀是动态的
```

在上面的例子中：
- `gensym("internal_temp")` 创建一个像 `##internal_temp#123` 这样的符号（确切的数字后缀可能会有所不同）。
- 即使我们使用 `esc` 将这个 `gensym` 创建的变量赋值给调用者的作用域，它也不会与调用者作用域中可能存在的同名（不带 `##` 和数字后缀）变量冲突。

`gensym` 对于编写需要引入许多辅助变量的复杂宏至关重要，同时要保持代码的健壮性并避免名称冲突。

与 `esc` 结合使用时，`gensym` 允许宏编写者精确控制哪些标识符是卫生的，哪些与调用者作用域交互，以及如何安全地引入新的、唯一的绑定到生成的代码中。
